Name: Yunyi Zhang<br>
Github Username: yunyizyz<br>
USC ID: 7518630167

In [102]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

**Data Preparation**

In [103]:
#Test Data
#CR: Chatgpt
test_dfs = []

# Bending1 test datasets
for i in range(1, 3):
    file_path = f'../AReM/bending1/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    test_dfs.append(df)

# Bending2 test datasets
for i in range(1, 3):
    file_path = f'../AReM/bending2/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    test_dfs.append(df)

# Cycling test datasets
for i in range(1, 4):
    file_path = f'../AReM/cycling/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    test_dfs.append(df)

# Lying test datasets
for i in range(1, 4):
    file_path = f'../AReM/lying/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    test_dfs.append(df)

# Sitting test datasets
for i in range(1, 4):
    file_path = f'../AReM/sitting/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    test_dfs.append(df)

# Standing test datasets
for i in range(1, 4):
    file_path = f'../AReM/standing/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    test_dfs.append(df)

# Walking test datasets
for i in range(1, 4):
    file_path = f'../AReM/walking/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    test_dfs.append(df)

# Concatenate all DataFrames into one testing dataset
test_data = pd.concat(test_dfs, ignore_index=True)

In [104]:
#Training Data
dfs = []

# Bending1 datasets
for i in range(3, 8):
    file_path = f'../AReM/bending1/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    dfs.append(df)

# Bending2 datasets
for i in range(3, 7):
    file_path = f'../AReM/bending2/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    dfs.append(df)

# Cycling datasets
for i in range(4, 16):
    file_path = f'../AReM/cycling/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    dfs.append(df)

# Lying datasets
for i in range(4, 16):
    file_path = f'../AReM/lying/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    dfs.append(df)

# Sitting datasets
for i in range(4, 16):
    file_path = f'../AReM/sitting/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    dfs.append(df)

# Standing datasets
for i in range(4, 16):
    file_path = f'../AReM/standing/dataset{i}.csv'
    df = pd.read_csv(file_path, skiprows=4, header=0)
    dfs.append(df)

# Concatenate all DataFrames into one training dataset
training_data = pd.concat(dfs, ignore_index=True)


In [105]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9120 entries, 0 to 9119
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   # Columns: time  9120 non-null   int64  
 1   avg_rss12        9120 non-null   float64
 2   var_rss12        9120 non-null   float64
 3   avg_rss13        9120 non-null   float64
 4   var_rss13        9120 non-null   float64
 5   avg_rss23        9120 non-null   float64
 6   var_rss23        9120 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 498.9 KB


In [106]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27359 entries, 0 to 27358
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   # Columns: time  27359 non-null  int64  
 1   avg_rss12        27359 non-null  float64
 2   var_rss12        27359 non-null  float64
 3   avg_rss13        27359 non-null  float64
 4   var_rss13        27359 non-null  float64
 5   avg_rss23        27359 non-null  float64
 6   var_rss23        27359 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 1.5 MB


 **Feature Extraction**<br><br>
 Classification of time series usually needs extracting features from them. In this problem, we focus on time-domain features.<br>
i. Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

**Time-domain features**<br><br>
Mean: The average value of the time series data over a given interval.<br>
Median: The middle value in a sorted list of the time series data.<br>
Standard deviation: A measure of the amount of variation or dispersion in the time series data.<br>
Variance: A measure of how spread out the values are in the time series data.<br>
Minimum: The smallest value in the time series data.<br>
Maximum: The largest value in the time series data.<br>
Range: The difference between the maximum and minimum values.<br>
Skewness: A measure of the asymmetry of the distribution of values in the time series.<br>
Kurtosis: A measure of the "tailedness" or sharpness of the distribution of values in the time series.<br>
Zero-crossings: The number of times the time series crosses the zero value.<br>
Quantiles: Values that divide the data into specified fractions, such as quartiles (Q1, Q2, Q3).<br>
Root mean square: The square root of the mean of the squared values in the time series.<br>
Interquartile range (IQR): The range between the first quartile (Q1) and the third quartile (Q3).<br>
Mean absolute deviation (MAD): The average of the absolute differences between each data point and the mean.<br>

ii. Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly.

In [107]:
# Function to calculate time-domain features
def calculate_time_domain_features(data):
    features = {
        'minimum': np.min(data),
        'maximum': np.max(data),
        'mean': np.mean(data),
        'median': np.median(data),
        'std_dev': np.std(data),
        'first_quartile': np.percentile(data, 25),
        'third_quartile': np.percentile(data, 75)
    }
    return features

# Directory paths for the datasets
dataset_paths = {
    'bending1': '../AReM/bending1/',
    'bending2': '../AReM/bending2/',
    'cycling': '../AReM/cycling/',
    'lying': '../AReM/lying/',
    'sitting': '../AReM/sitting/',
    'standing': '../AReM/standing/',
    'walking': '../AReM/walking/'
}

In [108]:
# Initialize a DataFrame to store the summary
summary_df = pd.DataFrame(columns=['Dataset', 'Feature_Raw Data', 'Value'])

# Process each dataset
for activity, path in dataset_paths.items():
    num_files = 7 if activity == 'bending1' else 6 if activity == 'bending2' else 15
    for i in range(1, num_files + 1):  # Adjusted range for bending1 and bending2
        file_path = os.path.join(path, f'dataset{i}.csv')
        df = pd.read_csv(file_path, skiprows=4, header=0)
        
        for j in range(1, 7):
            features = calculate_time_domain_features(df.iloc[:, j])
            for feature_name, value in features.items():
                row = [f'{activity}_dataset{i}', f'{j}_{feature_name}', value]
                summary_df.loc[len(summary_df)] = row

In [109]:
# Pivoting the DataFrame and setting 'Dataset' as the index
summary_pivot = summary_df.pivot(index='Dataset', columns='Feature_Raw Data', values='Value').reset_index()

# Display the pivoted summary table with 'Dataset' as index
summary_pivot.set_index('Dataset', inplace=True)

# Display the pivoted summary table
summary_pivot

Feature_Raw Data,1_first_quartile,1_maximum,1_mean,1_median,1_minimum,1_std_dev,1_third_quartile,2_first_quartile,2_maximum,2_mean,...,5_minimum,5_std_dev,5_third_quartile,6_first_quartile,6_maximum,6_mean,6_median,6_minimum,6_std_dev,6_third_quartile
Dataset,,,,,,,,,,,,,,,,,,,,,
bending1_dataset1,39.25,45.00,40.624792,40.500,37.25,1.475428,42.0000,0.00,1.30,0.358604,...,27.25,2.186168,36.00,0.00,1.92,0.570583,0.430,0.0,0.582308,1.3000
bending1_dataset2,42.00,45.67,42.812812,42.500,38.00,1.434054,43.6700,0.00,1.22,0.372437,...,27.67,1.993175,34.50,0.00,3.11,0.571083,0.430,0.0,0.600383,1.3000
bending1_dataset3,43.00,47.40,43.954500,44.330,35.00,1.557210,45.0000,0.00,1.70,0.426250,...,29.00,1.997520,36.50,0.00,1.79,0.493292,0.430,0.0,0.512971,0.9400
bending1_dataset4,39.15,47.75,42.179812,43.500,33.00,3.666840,45.0000,0.00,3.00,0.696042,...,20.00,3.845436,36.33,0.00,2.18,0.613521,0.500,0.0,0.523771,1.0000
bending1_dataset5,41.33,45.75,41.678063,41.750,33.00,2.241152,42.7500,0.43,2.83,0.535979,...,23.67,2.408514,31.25,0.00,1.79,0.383292,0.430,0.0,0.388759,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
walking_dataset5,31.67,46.25,34.763333,35.290,20.75,4.737266,38.2500,2.49,12.68,4.223792,...,6.00,3.171372,18.33,2.05,9.39,3.288271,3.270,0.0,1.645811,4.3050
walking_dataset6,32.00,51.00,34.935812,35.500,21.50,4.641102,38.0625,2.35,12.21,4.115750,...,6.33,3.188731,18.25,2.12,10.21,3.280021,3.015,0.0,1.699145,4.5000
walking_dataset7,31.25,47.67,34.333042,34.750,18.33,4.943612,38.0000,2.49,12.48,4.396958,...,6.67,2.997366,18.00,2.05,8.01,3.261583,2.980,0.0,1.615604,4.3200


In [110]:
# Exploraion for normalized data
summary_df_nrm = pd.DataFrame(columns=['Dataset', 'Feature_Normalized Data', 'Value'])

# Process each dataset
for activity, path in dataset_paths.items():
    num_files = 7 if activity == 'bending1' else 6 if activity == 'bending2' else 15
    for i in range(1, num_files + 1):  # Adjusted range for bending1 and bending2
        file_path = os.path.join(path, f'dataset{i}.csv')
        df = pd.read_csv(file_path, skiprows=4, header=0)

        # Normalize the raw data in each column
        scaler = MinMaxScaler()
        df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:])
        
        for j in range(1, 7):
            features = calculate_time_domain_features(df.iloc[:, j])
            for feature_name, value in features.items():
                row = [f'{activity}_dataset{i}', f'{j}_{feature_name}', value]
                summary_df_nrm.loc[len(summary_df_nrm)] = row

# Pivoting the DataFrame and setting 'Dataset' as the index
summary_pivot_nrm = summary_df_nrm.pivot(index='Dataset', columns='Feature_Normalized Data', values='Value').reset_index()

# Display the pivoted summary table with 'Dataset' as index
summary_pivot_nrm.set_index('Dataset', inplace=True)

# Display the pivoted summary table
summary_pivot_nrm

Feature_Normalized Data,1_first_quartile,1_maximum,1_mean,1_median,1_minimum,1_std_dev,1_third_quartile,2_first_quartile,2_maximum,2_mean,...,5_minimum,5_std_dev,5_third_quartile,6_first_quartile,6_maximum,6_mean,6_median,6_minimum,6_std_dev,6_third_quartile
Dataset,,,,,,,,,,,,,,,,,,,,,
bending1_dataset1,0.258065,1.0,0.435457,0.419355,0.0,0.190378,0.612903,0.000000,1.0,0.275849,...,0.0,0.198743,0.795455,0.000000,1.0,0.297179,0.223958,0.0,0.303285,0.677083
bending1_dataset2,0.521512,1.0,0.627485,0.586701,0.0,0.186969,0.739244,0.000000,1.0,0.305277,...,0.0,0.184042,0.630656,0.000000,1.0,0.183628,0.138264,0.0,0.193049,0.418006
bending1_dataset3,0.645161,1.0,0.722137,0.752419,0.0,0.125581,0.806452,0.000000,1.0,0.250735,...,0.0,0.210265,0.789474,0.000000,1.0,0.275582,0.240223,0.0,0.286576,0.525140
bending1_dataset4,0.416949,1.0,0.622360,0.711864,0.0,0.248599,0.813559,0.000000,1.0,0.232014,...,0.0,0.205969,0.874665,0.000000,1.0,0.281432,0.229358,0.0,0.240262,0.458716
bending1_dataset5,0.653333,1.0,0.680632,0.686275,0.0,0.175777,0.764706,0.151943,1.0,0.189392,...,0.0,0.174151,0.548084,0.000000,1.0,0.214129,0.240223,0.0,0.217184,0.279330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
walking_dataset5,0.428235,1.0,0.549542,0.570196,0.0,0.185775,0.686275,0.196372,1.0,0.333107,...,0.0,0.162634,0.632308,0.218317,1.0,0.350189,0.348243,0.0,0.175273,0.458466
walking_dataset6,0.355932,1.0,0.455451,0.474576,0.0,0.157326,0.561441,0.192465,1.0,0.337080,...,0.0,0.170794,0.638457,0.207640,1.0,0.321256,0.295299,0.0,0.166420,0.440744
walking_dataset7,0.440354,1.0,0.545434,0.559646,0.0,0.168494,0.670416,0.199519,1.0,0.352320,...,0.0,0.172958,0.653780,0.255930,1.0,0.407189,0.372035,0.0,0.201698,0.539326


iii. Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.

In [111]:
def bootstrap_ci(data, num_bootstrap_samples=1000, alpha=0.1):
    bootstrap_samples = np.zeros(num_bootstrap_samples)
    for i in range(num_bootstrap_samples):
        bootstrap_sample = resample(data)
        bootstrap_samples[i] = np.std(bootstrap_sample)
    
    lower_bound = np.percentile(bootstrap_samples, 100 * alpha / 2)
    upper_bound = np.percentile(bootstrap_samples, 100 * (1 - alpha / 2))
    return lower_bound, upper_bound

In [112]:
# Dictionary to store standard deviations and confidence intervals
feature_std_ci = {}

# Process each feature
for feature in summary_pivot.columns:
    if feature != 'Dataset':
        feature_data = summary_pivot[feature].dropna()
        std = np.std(feature_data)
        lower_bound, upper_bound = bootstrap_ci(feature_data)
        feature_std_ci[feature] = {'std': std, 'ci_lower': lower_bound, 'ci_upper': upper_bound}

df2 = pd.DataFrame(feature_std_ci).T.reset_index()
df2.columns = ['Feature', 'Standard Deviation', '90% Bootstrap CI (Lower)', '90% Bootstrap CI (Upper)']


In [113]:
df2

,Feature,Standard Deviation,90% Bootstrap CI (Lower),90% Bootstrap CI (Upper)
0,1_first_quartile,6.118526,5.497816,6.621139
1,1_maximum,4.369322,3.247486,5.200328
2,1_mean,5.305314,4.665995,5.830754
3,1_median,5.409056,4.763002,5.964417
4,1_minimum,9.515445,8.233632,10.745014
5,1_std_dev,1.760219,1.560959,1.953099
6,1_third_quartile,5.109643,4.304421,5.844017
7,2_first_quartile,0.940994,0.826213,1.026841
8,2_maximum,5.033882,4.575315,5.357474
9,2_mean,1.565194,1.398253,1.704423


Use your judgement to select the three most important time-domain features (one option may be min, mean, and max)

I think mean, standard deviation and median are the most important time-domain features. Because these three features can show the overall distribution of the value.

**2. ISLR 3.7.4**

(a) In this case, I would expect **cubic regression has a lower RSS**. Because although X and y are like linear relationship, a more flexible model like cubic model can still generate a lower RSS because it can fit the data better. <br> However, this is in the normal situation. If we consider X and y are EXACTLY linear relationship without any noise, the RSS should be both zero.

(b) In this case, I would expect **linear regression has a lower RSS**. Because cubic regression may overfit the training data and include some unrelated factors.

(c) In this case, I would expect the **cubic model has a lower RSS** because it is more flexible. When the relationship is not linear and we don't know how the distribution is, a more flexible model would result in lower RSS because it can better fit the data.

(d) In this case, I think we need more information to judge. This depend on how far it is from linear regression. If it is almost like linear regression, then linear model may have a lower test RSS. If it is very far from linear, then the cubic model may have a lower RSS.